# Project Goals
## Overview
## The Problem: Finding a Needle in a Haystack

# The Data: First Look

Let's get our data into a dataframe to examin it:

In [8]:
import src.wrangle
df = src.wrangle.get_raw_data()

## Size of the Raw Data

In [7]:
print(f'''Number of Columns: {df.shape[1]}, Number of Rows: {df.shape[0]}''')

Number of Columns: 185, Number of Rows: 91713


> This means that we have *91,713* patients in our dataset, with *185* metrics recorded for each of them within that 24 hour period. That's quite a bit of data.

## Imbalanced Data

Our goal for this project is to predict patient survivability, so what does that distribution look like in our data?

In [13]:
num_patients_died = len(df[df.hospital_death == 1])
print('Percentage of patients who did not survive: {: .2f}'.format(num_patients_died / len(df)))

Percentage of patients who did not survive:  0.09


>*91* percent of the patients survived their time in the ICU, while only *9* percent did not. 

While this is great news in terms of ICU survival rates, it means we're dealing with an imbalanced dataset which is going to raise its own challenges as we move forward.

# Preparation: Challenges Faced and How We Handled Them

## Handling Null Values in an Imbalanced Data Set

# Exploration: Examining Which Factors are Affecting Patient Survival
## Are certain hospitals better at data collection, and does that have an impact on patient outcomes?
## Is there a link between hospital and death rate?
## Does the age of the patient have a significant impact on patient survivability?
## The gender of the patient and outcome?

# Feature Engineering: 'Bringing a Magnet to the Needle in a Haystack Game'

# Modeling: Bringing it All Together

# Conclusion: What We've Done and What's Next